In [1]:
import random
import numpy as np
import pandas as pd

定义参数

In [34]:
num_steps = 10

In [35]:
raw_data = pd.read_csv("./Simulator/data/sim_train.csv", index_col="时间戳")
raw_data

,A磨煤机电流,A磨煤机进口与分离器出口差压,A磨煤机入口一次风风量2,A磨煤机入口热风隔绝门前一次风压力,A磨煤机入口密封风压力,A磨煤机密封风与一次风差压,A磨出口风粉混合物温度,3#A给煤机煤量(瞬时）,A磨煤机入口一次风风温,A磨煤机出口风压,...,过热器右侧一级喷水减温调节电动门开度反馈,过热器左侧一级喷水减温调节电动门开度反馈,过热器左侧二级喷水减温调节电动门开度反馈,过热器右侧二级喷水减温调节电动门开度反馈,引风机动叶开度反馈,一次风机入口开度反馈,送风机动叶开度反馈,一次风机速度反馈,左侧空预器冷端温度调节阀开度反馈,右侧空预器冷端温度调节阀开度反馈
时间戳,,,,,,,,,,,,,,,,,,,,,
1551283560,0.5770,0.5566,0.8461,0.6910,0.6603,0.5998,0.7169,0.7715,0.5454,0.2397,...,0.3886,0.0,0.0018,0.4116,0.2240,0.9972,0.0511,0.9148,0.0043,0.0036
1551283620,0.5625,0.5729,0.8584,0.6967,0.6673,0.5952,0.7157,0.7733,0.5420,0.2424,...,0.3886,0.0,0.0018,0.3981,0.2161,0.9973,0.0510,0.9158,0.0043,0.0036
1551283680,0.5738,0.5643,0.8563,0.6901,0.6631,0.5972,0.7136,0.7733,0.5408,0.2425,...,0.3886,0.0,0.0018,0.3710,0.2243,0.9973,0.0504,0.9130,0.0043,0.0036
1551283740,0.5763,0.5631,0.8490,0.6922,0.6619,0.5951,0.7124,0.7735,0.5408,0.2420,...,0.3886,0.0,0.0018,0.3710,0.2240,0.9973,0.0502,0.9147,0.0043,0.0036
1551283800,0.5718,0.5653,0.8597,0.6872,0.6677,0.6046,0.7121,0.7726,0.5399,0.2442,...,0.3886,0.0,0.0018,0.3710,0.2154,0.9972,0.0503,0.9185,0.0274,0.0302
1551283860,0.5557,0.5681,0.8578,0.6970,0.6629,0.5933,0.7122,0.7735,0.5389,0.2457,...,0.3886,0.0,0.0018,0.3710,0.2070,0.9973,0.0502,0.9201,0.0330,0.0309
1551283980,0.5911,0.5548,0.8589,0.6849,0.6579,0.5909,0.7112,0.7748,0.5399,0.2451,...,0.3989,0.0,0.0018,0.3710,0.2238,0.9972,0.0514,0.9175,0.0330,0.0309
1551284040,0.5612,0.5661,0.8583,0.6956,0.6624,0.5954,0.7128,0.7710,0.5374,0.2462,...,0.3989,0.0,0.0018,0.3777,0.2245,0.9973,0.0514,0.9212,0.0757,0.0145
1551284100,0.5535,0.5648,0.8640,0.7010,0.6609,0.5871,0.7120,0.7733,0.5422,0.2448,...,0.4247,0.0,0.0018,0.4068,0.2239,0.9973,0.0513,0.9199,0.0970,0.0145


In [36]:
class BoilerDataSet(object):
    """
    first run data_preparation.py to generate data.csv
    prepare boiler training and validation dataset
    simple version(small action dimension)

    """
    
    def __init__(self, num_steps, val_ratio=0.1):
        self.num_steps = num_steps  # 历史步长
        self.val_ratio = val_ratio  # 训练集与测试集比例
        
        # Read csv file
        self.raw_data = pd.read_csv("./Simulator/data/sim_train.csv", index_col='时间戳')

        # 划分训练集和测试集
        self.train_X, self.train_y, self.valid_X, self.valid_y = self.prepare_data(self.raw_data)

    def prepare_data(self, data):
        # split into groups of num_steps

        # 取出输入数据，学习num_steps步长的历史，iloc：通过行号获取行数据
        X = np.array([data.iloc[i: i + self.num_steps].values
                    for i in range(len(data) - self.num_steps)])

        # 取出输出数据，预测第num_steps步的值训练，ix / loc 可以通过行号和行标签进行索引
        # 这里只要对状态量进行预测即可，0-157列为 'A磨煤机电流':'大渣可燃物含量'
        y = np.array([data.iloc[i + self.num_steps, 0:158].values
                    for i in range(len(data) - self.num_steps)])

        train_size = int(len(X) * (1.0 - self.val_ratio))
        train_X, valid_X = X[:train_size], X[train_size:]
        train_y, valid_y = y[:train_size], y[train_size:]
        return train_X, train_y, valid_X, valid_y

    def generate_one_epoch(self, data_X, data_y, batch_size):
        num_batches = int(len(data_X)) // batch_size
        # if batch_size * num_batches < len(self.train_X):
        #     num_batches += 1

        batch_indices = list(range(num_batches))
        random.shuffle(batch_indices)
        for j in batch_indices:
            batch_X = data_X[j * batch_size: (j + 1) * batch_size]
            batch_y = data_y[j * batch_size: (j + 1) * batch_size]
            yield batch_X, batch_y

In [40]:
# read data 读入数据
boiler_dataset = BoilerDataSet(num_steps=num_steps, val_ratio=0.2)
train_X, train_y = boiler_dataset.train_X, boiler_dataset.train_y
valid_X, valid_y = boiler_dataset.valid_X, boiler_dataset.valid_y

在我们的示例中，一共提供了20组数据，设置的时间步长为10.因此，分别有从[0:9]->[10], [1:10]->[11], ... , [9:18]->[19] 共十组（X，y）\\
其中，我们训练集和测试集的比例为8：2，所以其中训练集有8组，测试集有2组。train_X (8, 10, 202) train_y()

In [41]:
# 打印数据信息
print('train samples: {0}'.format(len(train_X)))
print('valid samples: {0}'.format(len(valid_X)))

train samples: 8
eval  samples: 2


In [42]:
valid_y

array([[5.551e-01, 5.569e-01, 8.573e-01, 6.994e-01, 6.649e-01, 5.949e-01,
        7.114e-01, 7.716e-01, 5.319e-01, 2.570e-01, 7.635e-01, 6.819e-01,
        8.223e-01, 6.894e-01, 6.523e-01, 4.130e-01, 4.113e-01, 8.086e-01,
        7.281e-01, 3.732e-01, 6.222e-01, 5.793e-01, 8.053e-01, 6.559e-01,
        6.804e-01, 4.899e-01, 6.063e-01, 8.794e-01, 7.037e-01, 2.560e-01,
        6.894e-01, 5.938e-01, 8.801e-01, 6.546e-01, 6.767e-01, 5.099e-01,
        6.169e-01, 8.551e-01, 6.421e-01, 2.232e-01, 5.000e-04, 1.000e-04,
        2.030e-01, 6.580e-01, 2.349e-01, 5.000e-03, 5.460e-01, 2.100e-03,
        1.838e-01, 3.573e-01, 3.544e-01, 5.708e-01, 4.739e-01, 6.447e-01,
        5.650e-01, 5.685e-01, 5.398e-01, 4.602e-01, 3.734e-01, 7.474e-01,
        4.584e-01, 4.752e-01, 5.939e-01, 4.955e-01, 7.668e-01, 3.609e-01,
        7.235e-01, 3.782e-01, 2.814e-01, 2.441e-01, 3.143e-01, 4.387e-01,
        4.046e-01, 4.165e-01, 3.173e-01, 3.147e-01, 2.063e-01, 4.445e-01,
        4.092e-01, 4.383e-01, 7.295e-0